In [44]:
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [146]:
submissions_path = "Data"
all_files = os.listdir(submissions_path)

In [147]:
all_files

['2-level_stacked_1.csv',
 '5foldsv1.csv',
 '5foldsv2.csv',
 'automl_porto.csv',
 'Froza_and_Pascal_LB284.csv',
 'gbm_submit.csv',
 'gpari.csv',
 'gpx.csv',
 'LGB_282.csv',
 'LightGBM.csv',
 'median_LB285.csv',
 'mix_log3_LB286.csv',
 'mix_log_LB286.csv',
 'mix_rank_LB287.csv',
 'my_xgb_LB284.csv',
 'NN.csv',
 'rgf_submit_LB282.csv',
 'single_xgb_LB284.csv',
 'stacked_1_LB284.csv',
 'submission_5_xgb.csv',
 'sub_harmonic_LB286.csv',
 'xgb_submit_LB284.csv']

In [164]:
files = [
 'Froza_and_Pascal_LB284.csv',
 'gpari.csv',
 'rgf_submit_LB282.csv',
# 'stacked_1_LB284.csv',
# 'my_xgb_LB284.csv',
 'xgb_submit_LB284.csv',
 'NN.csv',
 'gpx.csv',
 '2-level_stacked_1.csv'
  ]

In [165]:
outs = [pd.read_csv(os.path.join(submissions_path, f), index_col=0)\
        for f in files]

In [166]:
concat_df = pd.concat(outs, axis=1)
cols = list(map(lambda x: str(x), files))
concat_df.columns = cols

In [167]:
#concat_df.head()

In [168]:
concat_df["target"] = (concat_df.rank() / concat_df.shape[0]).mean(axis=1)

In [169]:
concat_df.head()

,Froza_and_Pascal_LB284.csv,gpari.csv,rgf_submit_LB282.csv,xgb_submit_LB284.csv,NN.csv,gpx.csv,2-level_stacked_1.csv,target
id,,,,,,,,
0,0.025818,0.397413,0.023685,0.039368,0.02501,0.025212,0.028746,0.309060
1,0.023008,0.391664,0.024046,0.039615,0.02569,0.024342,0.027267,0.285808
2,0.022168,0.378311,0.017658,0.037616,0.02301,0.022366,0.026056,0.220196
3,0.014779,0.257092,0.014766,0.022206,0.01587,0.013257,0.021889,0.042106
4,0.032938,0.483567,0.034534,0.054306,0.03361,0.030355,0.035203,0.546685


In [170]:
concat_df.rank().head()

,Froza_and_Pascal_LB284.csv,gpari.csv,rgf_submit_LB282.csv,xgb_submit_LB284.csv,NN.csv,gpx.csv,2-level_stacked_1.csv,target
id,,,,,,,,
0,289668.0,260559.0,231234.0,265416.5,250297.0,284551.5,349807.0,270768.0
1,214537.0,247465.5,240710.0,269826.5,268218.5,262133.5,283324.0,249614.0
2,192900.0,217429.5,84098.0,235132.0,199231.0,212250.5,235119.0,190232.0
3,42394.0,29029.0,34169.0,27952.0,49117.5,31447.0,49040.0,32219.0
4,474223.5,474164.5,500160.0,515593.5,473614.5,416904.5,561965.0,490058.0


In [171]:
concat_df.drop(files, axis=1, inplace=True)
concat_df.to_csv("mix_m7_v2_1stack.csv")

In [14]:
concat_df["NN.csv"] -= 0.001

In [15]:
ALPHA = 5e-3  # Regularizaiton parameter
import numpy as np
from scipy.special import expit # sigmoid

logits = concat_df.applymap(lambda x: np.log(x/(1-x)))
logits *= (1 - ALPHA*logits**2)
stdevs = logits.std()  
w = .2/stdevs
wa = (w*logits).sum(axis=1)/w.sum()

# Convert back to probabilities
result = wa.apply(expit)
print( result.head() )

id
0    0.053262
1    0.051938
2    0.047649
3    0.034272
4    0.068777
dtype: float64


In [16]:
concat_df["y"] = result

In [17]:
concat_df.rank().head()

,Froza_and_Pascal_LB284.csv,gpari.csv,rgf_submit_LB282.csv,stacked_1_LB284.csv,my_xgb_LB284.csv,NN.csv,target,y
id,,,,,,,,
0,289668.0,260559.0,231234.0,377691.0,264508.0,250297.0,274660.0,269585.0
1,214537.0,247465.5,240710.0,286706.0,270420.5,268218.5,249834.0,249157.0
2,192900.0,217429.5,84098.0,242929.0,235393.5,199231.0,189700.0,185059.0
3,42394.0,29029.0,34169.0,47349.0,28573.5,49117.5,33430.0,33735.0
4,474223.5,474164.5,500160.0,549887.0,520755.5,473614.5,500678.0,498087.0


In [22]:
sub = concat_df[["y"]]
sub.columns = ["target"]
sub.to_csv("mix_m5_NN_logit.csv")

In [82]:
concat_df.drop(files, axis=1, inplace=True)

In [83]:
concat_df.to_csv("./mix_m4_NN.csv")

In [72]:
concat_df.rank().head()

,Froza_and_Pascal_LB284.csv,gpari.csv,rgf_submit_LB282.csv,stacked_1_LB284.csv,my_xgb_LB284.csv,NN.csv,target
id,,,,,,,
0,289668.0,260559.0,231234.0,377691.0,264508.0,250297.0,274660.0
1,214537.0,247465.5,240710.0,286706.0,270420.5,268218.5,249834.0
2,192900.0,217429.5,84098.0,242929.0,235393.5,199231.0,189700.0
3,42394.0,29029.0,34169.0,47349.0,28573.5,49117.5,33430.0
4,474223.5,474164.5,500160.0,549887.0,520755.5,473614.5,500678.0


In [51]:
concat_df.rank().head()

,Froza_and_Pascal_LB284.csv,gpari.csv,rgf_submit_LB282.csv,my_xgb_LB284.csv,stacked_1_LB284.csv,target
id,,,,,,
0,289668.0,260559.0,231234.0,264508.0,377691.0,280970.0
1,214537.0,247465.5,240710.0,270420.5,286706.0,247629.0
2,192900.0,217429.5,84098.0,235393.5,242929.0,189259.0
3,42394.0,29029.0,34169.0,28573.5,47349.0,31722.0
4,474223.5,474164.5,500160.0,520755.5,549887.0,505980.0


In [25]:
ff.shape

(595211, 2)

In [24]:
ff.sort_values(by="id")

,id,target
0,7,0.040778
1,9,0.035454
2,13,0.019035
3,16,0.017758
4,17,0.034774
5,19,0.034291
6,20,0.019592
7,22,0.023732
8,26,0.027421
9,28,0.048117
